In [2]:
!pip -q install langchain tiktoken duckduckgo-search

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.7/806.7 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 68.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 53.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 237.0/237.0 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 52.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.


In [3]:
import os

os.environ["OPENAI_API_KEY"] = ""

In [4]:
!pip show langchain

Name: langchain
Version: 0.1.5
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, async-timeout, dataclasses-json, jsonpatch, langchain-community, langchain-core, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


## Custom Agents with Custom Search




In [5]:
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.prompts import StringPromptTemplate

from langchain import OpenAI, LLMChain
from langchain.tools import DuckDuckGoSearchRun

from typing import List, Union
from langchain.schema import AgentAction, AgentFinish
import re
import langchain

## Setup Tools

In [17]:
# Define which tools the agent can use to answer user queries
search = DuckDuckGoSearchRun()

tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to answer questions about current events"
    )
]

In [7]:
obj = search.run("How can I treat a sprained ankle?")
type(obj)

/usr/local/lib/python3.10/dist-packages/langchain_community/utilities/duckduckgo_search.py:47: UserWarning: DDGS running in an async loop. This may cause errors. Use AsyncDDGS instead.
  with DDGS() as ddgs:


str

In [8]:
obj

'Compression Using a compression sleeve, elastic bandage wrap or brace on the affected ankle can prevent excessive swelling, protect the area from repeated injuries and provide support. Some inflammation is needed for healing, but too much can be harmful. Elevation The early treatment for an ankle sprain is the "RICE" method of treatment. This is focused on reducing swelling and reducing pain: Rest: The first 24 to 48 hours after the injury is considered a critical treatment period. Limit activities and use an assistive device, like a crutch, if you must walk. Health Library / Diseases & Conditions / Sprains Sprains Sprains are common injuries. They happen when ligaments around one of your joints are stretched or torn. Sport injuries and falls are the most common causes. I ce: Using ice as a sprain treatment can help reduce both swelling and pain, but too much ice can slow down the healing process, so use sparingly. C ompression: Wrap the injured area in an Ace bandage to help flush ou

In [9]:
search.run("site:webmd.com How can I treat a sprained ankle?")

/usr/local/lib/python3.10/dist-packages/langchain_community/utilities/duckduckgo_search.py:47: UserWarning: DDGS running in an async loop. This may cause errors. Use AsyncDDGS instead.
  with DDGS() as ddgs:


'11 /14. This stands for "rest," "ice," "compression," and "elevation.". It\'s most useful for sprains and broken bones, but it can help if your feet are swollen for other reasons ... Days 1-2: Recovery Begins. 3 /10. Your medical team will ask you to get up and start moving as soon as you can. You\'ll need to use crutches or a walker, because you have to keep your weight off ... 1 /19. Deep vein thrombosis is a blood clot that forms inside a vein, usually deep within your leg. As many as 900,000 Americans a year get one, and up to 100,000 die because of it. The danger is ... You treat it the same way: Rest it, ice it, raise it, and use pain relievers. When to Call the Doctor. 8/13. A bruise can need medical care if: You think a sprain or broken bone may have caused it. 2 /23. Ultra-high heels force the feet into a position that puts stress on the ball of the foot. At this joint, the long metatarsal bones meet the pea-shaped sesamoid bones and the toe bones ...'

In [10]:
def duck_wrapper(input_text):
    search_results = search.run(f"site:webmd.com {input_text}")
    return search_results

tools = [
    Tool(
        name = "Search WebMD",
        func=duck_wrapper,
        description="useful for when you need to answer medical and pharmalogical questions"
    )
]

## Prompt Template
This instructs the agent on what to do. Generally, the template should incorporate:

**`tools:`** which tools the agent has access and how and when to call them.

**`intermediate_steps:`** These are tuples of previous (**`AgentAction`**, **`Observation`**) pairs. These are generally not passed directly to the model, but the prompt template formats them in a specific way.

**`input:`** generic user input

In [11]:
# Set up the base template
template = """Answer the following questions as best you can, but speaking as compasionate medical professional. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! Remember to answer as a compansionate medical professional when giving your final answer.

Question: {input}
{agent_scratchpad}"""

In [12]:
# Set up a prompt template
class CustomPromptTemplate(StringPromptTemplate):
    # The template to use
    template: str
    # The list of tools available
    tools: List[Tool]

    def format(self, **kwargs) -> str:
        # Get the intermediate steps (AgentAction, Observation tuples)
        # Format them in a particular way
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        # Set the agent_scratchpad variable to that value
        kwargs["agent_scratchpad"] = thoughts
        # Create a tools variable from the list of tools provided
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
        # Create a list of tool names for the tools provided
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
        return self.template.format(**kwargs)

In [13]:
prompt = CustomPromptTemplate(
    template=template,
    tools=tools,
    # This omits the `agent_scratchpad`, `tools`, and `tool_names` variables because those are generated dynamically
    # This includes the `intermediate_steps` variable because that is needed
    input_variables=["input", "intermediate_steps"]
)

## Custom Output Parser
The output parser is responsible for parsing the LLM output into AgentAction and AgentFinish. This usually depends heavily on the prompt used.

This is where you can change the parsing to do retries, handle whitespace, etc

In [14]:
class CustomOutputParser(AgentOutputParser):

    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        # Check if agent should finish
        if "Final Answer:" in llm_output:
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        # Parse out the action and action input
        regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            raise ValueError(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        # Return the action and action input
        return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)

In [15]:
output_parser = CustomOutputParser()

## Set up LLM
Choose the LLM you want to use!

In [16]:
llm = OpenAI(temperature=0)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


ValidationError: 1 validation error for OpenAI
__root__
  Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. (type=value_error)

## Define the stop sequence
This is important because it tells the LLM when to stop generation.

This depends heavily on the prompt and model you are using. Generally, you want this to be whatever token you use in the prompt to denote the start of an Observation (otherwise, the LLM may hallucinate an observation for you).

## Set up the Agent
We can now combine everything to set up our agent

In [ ]:
# LLM chain consisting of the LLM and a prompt
llm_chain = LLMChain(llm=llm, prompt=prompt)

In [ ]:
tool_names = [tool.name for tool in tools]

agent = LLMSingleActionAgent(
    llm_chain=llm_chain,
    output_parser=output_parser,
    stop=["\nObservation:"],
    allowed_tools=tool_names
)


### What is an Agent Executor?

Agent Executors take an agent and tools and use the agent to decide which tools to call and in what order.



In [ ]:
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent,
                                                    tools=tools,
                                                    verbose=True)

In [ ]:
agent_executor.run("How can I treat a spained ankle?")



> Entering new AgentExecutor chain...


Thought: I need to find out what the best treatment is for a sprain.
Action: Search WebMD
Action Input: "sprained ankle treatment"

Observation:3 /12. To curb swelling and pain in the first 24 hours after a minor injury, apply a cold compress for 20-30 minutes. Then remove it for 20-30 minutes. You can use a bag of frozen peas. Or put ice ... Achilles Tendon Injury Symptoms. The most obvious sign is pain above your heel, especially when you stretch your ankle or stand on your toes. It may be mild and get better or worse over time. If ... Symptoms of muscle strain include: Swelling, bruising, or redness due to the injury. Pain at rest. Pain when the specific muscle or the joint in relation to that muscle is used. Weakness of the ... Trauma to an area (jerking movements, auto accidents, falls, fractures, sprains, dislocations, and direct blows to the muscle) also can cause musculoskeletal pain. Other causes of pain include ... Being overweight. Stress fractures in your toes or metatarsal

'To treat a sprain, it is best to apply a cold compress for 20-30 minutes and then remove it for 20-30 minutes. This will help reduce swelling and pain in the first 24 hours after the injury.'

### in Debug mode

In [ ]:
langchain.debug = True

In [ ]:
agent_executor.run("How can I treat a spained ankle?")

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "How can I treat a spained ankle?"
}
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
{
  "intermediate_steps": [],
  "stop": [
    "\nObservation:"
  ],
  "input": "How can I treat a spained ankle?"
}
[llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:OpenAI] Entering LLM run with input:
{
  "prompts": [
    "Answer the following questions as best you can, but speaking as compasionate medical professional. You have access to the following tools:\n\nSearch WebMD: useful for when you need to answer medical and pharmalogical questions\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [Search WebMD]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat 

[llm/end] [1:chain:AgentExecutor > 5:chain:LLMChain > 6:llm:OpenAI] [17.57s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": " I now know the best treatment for a sprain is to apply a cold compress for 20-30 minutes and then remove it for 20-30 minutes.\nFinal Answer: To treat a sprain, it is best to apply a cold compress for 20-30 minutes and then remove it for 20-30 minutes. This will help reduce swelling and pain in the first 24 hours after the injury.",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        }
      }
    ]
  ],
  "llm_output": {
    "token_usage": {
      "completion_tokens": 80,
      "prompt_tokens": 455,
      "total_tokens": 535
    },
    "model_name": "text-davinci-003"
  }
}
[chain/end] [1:chain:AgentExecutor > 5:chain:LLMChain] [17.57s] Exiting Chain run with output:
{
  "text": " I now know the best treatment for a sprain is to apply a cold compress for 20-30 minutes and then remov

'To treat a sprain, it is best to apply a cold compress for 20-30 minutes and then remove it for 20-30 minutes. This will help reduce swelling and pain in the first 24 hours after the injury.'

In [ ]:
langchain.debug = False

## Adding a Conversation Memory
If you want to add memory to the agent, you’ll need to:

1. Add  **`chat_history`** into the custom prompt

2. Add the memory object to the agent executor when defining it.

In [ ]:
# Set up the base template
template_with_history = """Answer the following questions as best you can, but speaking as compasionate medical professional. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! Remember to speak as a compasionate medical professional when giving your final answer. If the condition is serious advise they speak to a doctor.

Previous conversation history:
{history}

New question: {input}
{agent_scratchpad}"""

In [ ]:
prompt_with_history = CustomPromptTemplate(
    template=template_with_history,
    tools=tools,
    # This omits the `agent_scratchpad`, `tools`, and `tool_names` variables because those are generated dynamically
    # This includes the `intermediate_steps` variable because that is needed
    input_variables=["input", "intermediate_steps", "history"]
)

In [ ]:
llm_chain = LLMChain(llm=llm, prompt=prompt_with_history)

In [ ]:
tool_names = [tool.name for tool in tools]
agent = LLMSingleActionAgent(
    llm_chain=llm_chain,
    output_parser=output_parser,
    stop=["\nObservation:"],
    allowed_tools=tool_names
)

In [ ]:
from langchain.memory import ConversationBufferWindowMemory

memory=ConversationBufferWindowMemory(k=2)

In [ ]:
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
    verbose=True,
    memory=memory
    )

In [ ]:
agent_executor.run("How can I treat a sprained ankle?")



> Entering new AgentExecutor chain...

Thought: I need to find out what the best treatment for a sprained ankle is.
Action: Search WebMD
Action Input: "sprained ankle treatment"

Observation:3 /12. To curb swelling and pain in the first 24 hours after a minor injury, apply a cold compress for 20-30 minutes. Then remove it for 20-30 minutes. You can use a bag of frozen peas. Or put ice ... Achilles Tendon Injury Symptoms. The most obvious sign is pain above your heel, especially when you stretch your ankle or stand on your toes. It may be mild and get better or worse over time. If ... Symptoms of muscle strain include: Swelling, bruising, or redness due to the injury. Pain at rest. Pain when the specific muscle or the joint in relation to that muscle is used. Weakness of the ... Trauma to an area (jerking movements, auto accidents, falls, fractures, sprains, dislocations, and direct blows to the muscle) also can cause musculoskeletal pain. Other causes of pain include ... Being overwe

'To treat a sprained ankle, it is important to reduce swelling and pain in the first 24 hours after the injury. Apply a cold compress for 20-30 minutes, then remove it for 20-30 minutes. You can use a bag of frozen peas. If the pain persists or worsens, it is important to seek medical attention.'

In [ ]:
agent_executor.run("What meds could I take?")



> Entering new AgentExecutor chain...


Thought: I need to know what medications are available to treat a sprained ankle.
Action: Search WebMD
Action Input: Sprained ankle medications

Observation:3 /12. To curb swelling and pain in the first 24 hours after a minor injury, apply a cold compress for 20-30 minutes. Then remove it for 20-30 minutes. You can use a bag of frozen peas. Or put ice ... Achilles Tendon Injury Symptoms. The most obvious sign is pain above your heel, especially when you stretch your ankle or stand on your toes. It may be mild and get better or worse over time. If ... Symptoms of muscle strain include: Swelling, bruising, or redness due to the injury. Pain at rest. Pain when the specific muscle or the joint in relation to that muscle is used. Weakness of the ... RICE. 2 /14. It stands for rest, ice, compress, and elevate, and it's the go-to treatment for minor sprains and bruising. Ice the injury as soon as possible and put pressure on it with an ... Trauma to an area (jerking movements, auto accidents,

 I now know the medications available to treat a sprained ankle.
Final Answer: To treat a sprained ankle, it is important to reduce swelling and pain in the first 24 hours after the injury. Apply a cold compress for 20-30 minutes, then remove it for 20-30 minutes. You can use a bag of frozen peas. Other medications that may be used to treat a sprained ankle include over-the-counter pain relievers such as ibuprofen or acetaminophen, and topical creams or gels that contain menthol or capsaicin. If the pain persists or worsens, it is important to seek medical attention.

> Finished chain.


'To treat a sprained ankle, it is important to reduce swelling and pain in the first 24 hours after the injury. Apply a cold compress for 20-30 minutes, then remove it for 20-30 minutes. You can use a bag of frozen peas. Other medications that may be used to treat a sprained ankle include over-the-counter pain relievers such as ibuprofen or acetaminophen, and topical creams or gels that contain menthol or capsaicin. If the pain persists or worsens, it is important to seek medical attention.'

In [ ]:
agent_executor.run("How long will it take to heal?")



> Entering new AgentExecutor chain...

Thought: I need to find out how long it takes for a sprained ankle to heal.
Action: Search WebMD
Action Input: Sprained ankle healing time

Observation:3 /12. To curb swelling and pain in the first 24 hours after a minor injury, apply a cold compress for 20-30 minutes. Then remove it for 20-30 minutes. You can use a bag of frozen peas. Or put ice ... Achilles Tendon Injury Symptoms. The most obvious sign is pain above your heel, especially when you stretch your ankle or stand on your toes. It may be mild and get better or worse over time. If ... Symptoms of muscle strain include: Swelling, bruising, or redness due to the injury. Pain at rest. Pain when the specific muscle or the joint in relation to that muscle is used. Weakness of the ... 7 /15. Over the next 4-21 days, you get a soft callus around the broken bone. This is when a substance called collagen moves in and slowly replaces the blood clot. The callus is stiffer than a ... Many Colors.

'It typically takes 4-21 days for a sprained ankle to heal. During this time, it is important to reduce swelling and pain by applying a cold compress for 20-30 minutes, then removing it for 20-30 minutes. If the pain persists or worsens, it is important to seek medical attention.'